# <u>**Welcome to Skyy-Med!**</u>

# Configure Jupyter, our environment, and install Detectron2!

There are different methods to install and it can get complicated to match everything up.

official documantation: https://detectron2.readthedocs.io/en/latest/tutorials/install.html

How to install in docker: https://medium.com/the-owl/install-detectron2-73d9447d3652

How to install in Ubuntu: https://medium.com/@muhammadtalha1726/detectron-2-installation-guide-d12f66e220bf

1. Choose Python3 as the kernel for Jupyter initially (we will create a custom environment later and update this)

2. Install CUDA (or in my case just make it accessible via PATH since its already installed as part of FLUXCORE)

In [ ]:
#### turn this into a bash script to autoconfigure Fluxcore servers on the fly ####

# verify the version of Ubuntu server is 20.04
lsb_release -a

# install the basics
apt install pip
apt install nano
apt install wget
python3 -m pip install --upgrade pip

#install Jupyter notebooks webserver (requires port mappping)
apt-get update
pip3 install notebook
nohup jupyter notebook --ip=0.0.0.0 --allow-root --no-browser > jupyter.log 2>&1 &

#verify version of cuda is 12+ , if existing and not 12+ purge Cuda
nvcc --version
apt purge cuda*

# install a newer version of Cuda (12.03 or similar)
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.1-1_all.deb
dpkg -i cuda-keyring_1.1-1_all.deb
apt-get update
apt-get -y install cuda-toolkit-12-4

# Change directory to /usr/local
cd /usr/local
# Add CUDA bin directory to the PATH variable in /etc/environment
echo 'PATH="/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/local/cuda-12.4/bin"' | tee -a /etc/environment
# Source the /etc/environment file to apply changes immediately
source /etc/environment # if this doesnt work the one below it will for containers
export PATH=$PATH:/usr/local/cuda-12.4/bin
# Check if nvcc is available by running nvcc --version
if ! command -v nvcc &> /dev/null
then
    echo "nvcc command not found. CUDA installation might be incomplete or the PATH is not properly configured."
    exit 1
fi
nvcc --version
cd ~

# import os
# # Add CUDA binary directory to the PATH
# os.environ['PATH'] += ':/usr/local/cuda/bin'
# # Add CUDA library directory to LD_LIBRARY_PATH
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64'
# # test Cuda
# !nvcc --version

# pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
pip3 install torch torchvision torchaudio
pip install opencv-python
pip install cython
pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"
python3 -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
#### run these from terminal not inside Jupyter ####

# check to see if Cuda is installed (there will be a cuda directory if its installed)
cd /usr/local
#if it exists already note the version in the files (12.3 is what I have already from FluxCore)
# add cuda to environment variables
sudo nano /etc/environment
# add these lines to file
PATH="/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/local/cuda-11.8/bin"
# reload environment variables (or reboot)
source /etc/environment
# verify it works by listing version
nvcc --version

3. Install miniconda (if it doesnt already exist) so we can use CONDA environments
   
       official documentation https://www.anaconda.com/products/distribution

In [ ]:
#### Run these from terminal not inside Jupyter ####

# see if miniconda is installed
conda --version
# if not installed install it now
# use control + c to skip to the bottom of the agreement in FLuxedge, then type yes
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
bash miniconda.sh
# activate an environment for our project
conda create -n skyymed01 python=3.7
conda activate skyymed01
# add the conda kernel for Jupyter
conda install ipykernel
# Register the skymed01 environment as a Jupyter kernel
python -m ipykernel install --user --name skyymed01 --display-name "Python (skyymed01)"
# Restart Jupyter, then select the new skymed kernel
jupyter notebook stop && jupyter notebook start
# once it reloads choose the skymed kernel (top right)

In [ ]:
# additional commands you may need in the future:

# uninstall Jupyter kernels
jupyter kernelspec uninstall <kernel name>
# uninstall pytorch and clean up conda
conda remove pytorch
conda clean --all

4. Test Conda and Cuda are accessible from Jupyter

In [ ]:
import os

# Add Conda binary directory to the PATH
os.environ['PATH'] += ':/home/sirpoopsalot/miniconda/bin'

# Add CUDA binary directory to the PATH
os.environ['PATH'] += ':/usr/local/cuda/bin'

# Add CUDA library directory to LD_LIBRARY_PATH
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

# test Conda
!conda info

# test Cuda
!nvcc --version

5. make sure conda commands are available inside Jupyter by cheking the version

In [ ]:
conda --version

6. make sure we are working from within our new environment

In [ ]:
import sys
print(sys.executable)

7. Install Pytorch in our conda environment using the correct cuda version in the command (this can take time, be patient for it to finish)

   This install is dependant on which version of Cuda we are running and the install script changes depending
   Watch this for future issues if the Cuda version gets upgraded!

In [ ]:
#### run this from terminal not from Jupyter ####

# install PyTorch
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

8. restart the kernel in Jupyter under the Kernel pulldown
9. Verify Torch is available by listing the version

In [ ]:
import torch
print(torch.__version__)
import sys
print(sys.path)

10. Install OpenCV

In [ ]:
pip install opencv-python

11. Install Cython

In [ ]:
pip install cython

12. Install COCO Python API

In [ ]:
pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"

13. Install detectron2

    see official documantation at https://detectron2.readthedocs.io/en/latest/tutorials/install.html

In [ ]:
!python3 -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

14. Verify we have Detectron2 CUDA and Pytorch installed and accessible to Jupyter (display the versions)

In [ ]:
pip install --upgrade jupyter ipywidgets

16. verify that we have GPU processing available to our model

   this will make sure our drivers and libraires are correctly set

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU is not available. Using CPU.")

# Trouble shooting
Verify the user for Jupyter

In [ ]:
import getpass

current_user = getpass.getuser()
print("Current user:", current_user)


2. show whats in PATH for Jupyter

In [ ]:
!echo $PATH